In [ ]:
#r "nuget: Jinaga, 1.1.2"
#r "nuget: Jinaga.UnitTest, 1.1.2"
#r "nuget: Jinaga.Notebooks, 1.1.0"

Installed Packages Jinaga, 1.1.2 Jinaga.Notebooks, 1.1.0 Jinaga.UnitTest, 1.1.2

In [ ]:
using Jinaga;
using Jinaga.Notebooks;
using Jinaga.UnitTest;

In [ ]:
[FactType("University.Student")]
public record Student(string publicKey);

[FactType("University.Organization")]
public record Organization(string identifier);

[FactType("University.Application")]
public record Application(Student student, Organization organization, DateTime appliedAt);

[FactType("University.Enrollment")]
public record Enrollment(Application application);

[FactType("University.Rejection")]
public record Rejection(Application application);

[FactType("University.Course")]
public record Course(Organization organization, string code, string name);

[FactType("University.Semester")]
public record Semester(Organization organization, int year, string term);

[FactType("University.Instructor")]
public record Instructor(Organization organization, string name);

[FactType("University.Offering")]
public record Offering(Course course, Semester semester, Instructor instructor, string days, string time);

[FactType("University.Registration")]
public record Registration(Enrollment enrollment, Offering offering);

[FactType("University.Drop")]
public record Drop(Registration registration);

[FactType("University.Fail")]
public record Fail(Registration registration, int grade);

[FactType("University.Complete")]
public record Complete(Registration registration, int grade);

In [ ]:
Renderer.RenderTypes(typeof(Student), typeof(Organization), typeof(Application), typeof(Enrollment), typeof(Rejection), typeof(Course), typeof(Semester), typeof(Instructor), typeof(Offering), typeof(Registration), typeof(Drop), typeof(Fail), typeof(Complete))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- University.Student -->
 
 University.Student 
 
 University.Student 
 
<!-- University.Organization -->
 
 University.Organization 
 
 University.Organization 
 
<!-- University.Application -->
 
 University.Application 
 
 University.Application 
 
<!-- University.Application->University.Student -->
 
 University.Application->University.Student 
 
 
 student 
 
<!-- University.Application->University.Organization -->
 
 University.Application->University.Organization 
 
 
 organization 
 
<!-- University.Enrollment -->
 
 University.Enrollment 
 
 University.Enrollment 
 
<!-- University.Enrollment->University.Application -->
 
 University.Enrollment->University.Application 
 
 
 application 
 
<!-- University.Rejection -->
 
 University.Rejection 
 
 University.Rejection 
 
<!-- University.Rejection->University.Application -->
 
 University.Rejection->University.Application 
 
 
 application 
 
<!-- University.Course -->
 
 University.Course 
 
 University.Course 
 
<!-- University.Course->University.Organization -->
 
 University.Course->University.Organization 
 
 
 organization 
 
<!-- University.Semester -->
 
 University.Semester 
 
 University.Semester 
 
<!-- University.Semester->University.Organization -->
 
 University.Semester->University.Organization 
 
 
 organization 
 
<!-- University.Instructor -->
 
 University.Instructor 
 
 University.Instructor 
 
<!-- University.Instructor->University.Organization -->
 
 University.Instructor->University.Organization 
 
 
 organization 
 
<!-- University.Offering -->
 
 University.Offering 
 
 University.Offering 
 
<!-- University.Offering->University.Course -->
 
 University.Offering->University.Course 
 
 
 course 
 
<!-- University.Offering->University.Semester -->
 
 University.Offering->University.Semester 
 
 
 semester 
 
<!-- University.Offering->University.Instructor -->
 
 University.Offering->University.Instructor 
 
 
 instructor 
 
<!-- University.Registration -->
 
 University.Registration 
 
 University.Registration 
 
<!-- University.Registration->University.Enrollment -->
 
 University.Registration->University.Enrollment 
 
 
 enrollment 
 
<!-- University.Registration->University.Offering -->
 
 University.Registration->University.Offering 
 
 
 offering 
 
<!-- University.Drop -->
 
 University.Drop 
 
 University.Drop 
 
<!-- University.Drop->University.Registration -->
 
 University.Drop->University.Registration 
 
 
 registration 
 
<!-- University.Fail -->
 
 University.Fail 
 
 University.Fail 
 
<!-- University.Fail->University.Registration -->
 
 University.Fail->University.Registration 
 
 
 registration 
 
<!-- University.Complete -->
 
 University.Complete 
 
 University.Complete 
 
<!-- University.Complete->University.Registration -->
 
 University.Complete->University.Registration 
 
 
 registration